<a href="https://colab.research.google.com/github/hirokimituya/book_create-profitable-AI-with-Python/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=42209f726e4c1bbf792838570daa9f4760072db8d0a6e0dd8b3c54dfb89eb43b
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3713 sha256=165f24b9645b7f33bc6130e1c774561f5385c457e92318c5031e7a0677365453
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    # url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    # r = requests.get(url)
    # with open('./data/data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4272,サンドラッグ,9989,小売業
4273,サックスバー ホールディングス,9990,小売業
4274,ジェコス,9991,卸売業
4277,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, min_price=100, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3085]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[50749]の株価情報を取得できなかったため、スキップしました。
経過時間: 392.9824929237366秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,2000600,757.400024
1,ＩＮＰＥＸ,1605,鉱業,12667900,2174.000000
2,安藤・間,1719,建設業,671200,1234.000000
3,コムシスホールディングス,1721,建設業,1126900,3150.000000
4,オリエンタル白石,1786,建設業,584600,331.000000
...,...,...,...,...,...
353,ＳＣＳＫ,9719,情報・通信業,832500,2551.500000
354,丹青社,9743,サービス業,574300,859.000000
355,ＮＳＤ,9759,情報・通信業,779800,2470.000000
356,ヤマダホールディングス,9831,小売業,4186400,464.600006


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,2000600,757.400024
1,ＩＮＰＥＸ,1605,鉱業,12667900,2174.000000
2,安藤・間,1719,建設業,671200,1234.000000
3,コムシスホールディングス,1721,建設業,1126900,3150.000000
4,オリエンタル白石,1786,建設業,584600,331.000000
...,...,...,...,...,...
249,電源開発,9513,電気・ガス業,668500,2345.000000
250,東京瓦斯,9531,電気・ガス業,1617100,3392.000000
251,大阪瓦斯,9532,電気・ガス業,1749700,2429.500000
252,ＳＣＳＫ,9719,情報・通信業,832500,2551.500000


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ＩＮＰＥＸ,1605,鉱業,12667900,2174.000000
1,安藤・間,1719,建設業,671200,1234.000000
2,コムシスホールディングス,1721,建設業,1126900,3150.000000
3,オリエンタル白石,1786,建設業,584600,331.000000
4,大林組,1802,建設業,2195500,1357.000000
...,...,...,...,...,...
206,電源開発,9513,電気・ガス業,668500,2345.000000
207,東京瓦斯,9531,電気・ガス業,1617100,3392.000000
208,大阪瓦斯,9532,電気・ガス業,1749700,2429.500000
209,ＳＣＳＫ,9719,情報・通信業,832500,2551.500000


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,3578500,792.700012
1,パーソルホールディングス,2181,サービス業,636100,2559.000000
2,システナ,2317,情報・通信業,1842500,284.000000
3,日本駐車場開発,2353,不動産業,6335700,226.000000
4,カカクコム,2371,サービス業,1003200,1676.500000
...,...,...,...,...,...
100,ＮＴＴデータグループ,9613,情報・通信業,4416900,1972.500000
101,アインホールディングス,9627,小売業,583100,4609.000000
102,丹青社,9743,サービス業,574300,859.000000
103,ＮＳＤ,9759,情報・通信業,779800,2470.000000


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,神戸物産,3038,卸売業,901200,3525.000000
1,コーエーテクモホールディングス,3635,情報・通信業,608900,2224.500000
2,アイスタイル,3660,情報・通信業,1651200,472.000000
3,三菱製紙,3864,パルプ・紙,1261800,502.000000
4,ｇｕｍｉ,3903,情報・通信業,1627700,566.000000
5,ソースネクスト,4344,情報・通信業,1353000,186.000000
6,そーせいグループ,4565,医薬品,902100,1638.000000
7,第一三共,4568,医薬品,5098200,4155.000000
8,ペプチドリーム,4587,医薬品,816300,1749.000000
9,サイバーエージェント,4751,サービス業,26553700,854.799988


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4267,タキヒヨー,9982,卸売業
4270,蔵王産業,9986,卸売業
4275,ヤマザワ,9993,小売業
4276,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 100円以上、5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[92649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9967]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,832900,405.0
1,大盛工業,1844,建設業,237600,202.0
2,クルーズ,2138,情報・通信業,392000,1003.0
3,クックパッド,2193,サービス業,233400,161.0
4,クシム,2345,情報・通信業,1398800,334.0
...,...,...,...,...,...
68,アコム,8572,その他金融業,1897400,359.0
69,トレイダーズホールディングス,8704,証券、商品先物取引業,454500,811.0
70,明治海運,9115,海運業,361800,802.0
71,日本テレホン,9425,情報・通信業,959400,550.0


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,832900,405.0
1,クルーズ,2138,情報・通信業,392000,1003.0
2,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,911000,692.0
3,アップルインターナショナル,2788,卸売業,856900,553.0
4,ポラリス・ホールディングス,3010,サービス業,1273000,196.0
5,ビューティ花壇,3041,卸売業,1409100,1798.0
6,ＴＨＥグローバル社,3271,不動産業,206500,440.0
7,ランシステム,3326,サービス業,458000,481.0
8,マリオン,3494,不動産業,391900,1383.0
9,クラウディアホールディングス,3607,繊維製品,533200,662.0


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,832900,405.0
1,アップルインターナショナル,2788,卸売業,856900,553.0
2,ポラリス・ホールディングス,3010,サービス業,1273000,196.0
3,ビューティ花壇,3041,卸売業,1409100,1798.0
4,ＴＨＥグローバル社,3271,不動産業,206500,440.0
5,ランシステム,3326,サービス業,458000,481.0
6,マリオン,3494,不動産業,391900,1383.0
7,ジェイ・エスコムホールディングス,3779,情報・通信業,1085700,165.0
8,パス,3840,情報・通信業,5608600,146.0
9,セプテーニ・ホールディングス,4293,サービス業,414800,416.0


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,大盛工業,1844,建設業,237600,202.0
1,クックパッド,2193,サービス業,233400,161.0
2,クシム,2345,情報・通信業,1398800,334.0
3,出前館,2484,情報・通信業,712600,381.0
4,イメージ ワン,2667,卸売業,509600,610.0
5,セリア,2782,小売業,345600,2229.0
6,ストリーム,3071,小売業,215200,114.0
7,ユナイテッド・スーパーマーケット・ホールディングス,3222,小売業,206100,1120.0
8,プレサンスコーポレーション,3254,不動産業,368400,1933.0
9,ｅｎｉｓｈ,3667,情報・通信業,311900,249.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,出前館,2484,情報・通信業,712600,381.0
1,イメージ ワン,2667,卸売業,509600,610.0
2,セリア,2782,小売業,345600,2229.0
3,ストリーム,3071,小売業,215200,114.0
4,ｅｎｉｓｈ,3667,情報・通信業,311900,249.0
5,リミックスポイント,3825,情報・通信業,272700,215.0
6,Ａｂａｌａｎｃｅ,3856,電気機器,1038900,3495.0
7,ユークス,4334,情報・通信業,648100,1650.0
8,ユニバーサルエンターテインメント,6425,機械,246800,2338.0
9,伊豆シャボテンリゾート,6819,サービス業,375000,240.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

,銘柄名,コード,業種
45,ルーデン・ホールディングス,1400,建設業
46,エムビーエス,1401,建設業
55,Ｌｉｂ Ｗｏｒｋ,1431,建設業
60,フィット,1436,建設業
65,ニッソウ,1444,建設業
...,...,...,...
4091,プログリット,9560,サービス業
4092,グラッドキューブ,9561,サービス業
4093,ビジネスコーチ,9562,サービス業
4094,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,429200,852.0
1,ジーエヌアイグループ,2160,医薬品,3909700,2140.0
2,ウェッジホールディングス,2388,その他金融業,386600,138.0
3,ベースフード,2936,食料品,638600,638.0
4,ツクルバ,2978,不動産業,890100,1267.0
...,...,...,...,...,...
74,ＧＥＮＯＶＡ,9341,サービス業,243800,1487.0
75,ｉｓｐａｃｅ,9348,サービス業,324000,1558.0
76,リニューアブル・ジャパン,9522,電気・ガス業,3594500,1165.0
77,Ｍ＆Ａ総研ホールディングス,9552,サービス業,706000,3345.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,3909700,2140.0
1,ウェッジホールディングス,2388,その他金融業,386600,138.0
2,ベースフード,2936,食料品,638600,638.0
3,ツクルバ,2978,不動産業,890100,1267.0
4,クリアル,2998,不動産業,228100,4870.0
5,アプリックス,3727,情報・通信業,1835400,223.0
6,イメージ情報開発,3803,情報・通信業,832600,537.0
7,エコモット,3987,情報・通信業,602000,768.0
8,シェアリングテクノロジー,3989,情報・通信業,688200,775.0
9,アララ,4015,情報・通信業,489500,828.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,3909700,2140.0
1,ベースフード,2936,食料品,638600,638.0
2,ツクルバ,2978,不動産業,890100,1267.0
3,アプリックス,3727,情報・通信業,1835400,223.0
4,イメージ情報開発,3803,情報・通信業,832600,537.0
5,エコモット,3987,情報・通信業,602000,768.0
6,シェアリングテクノロジー,3989,情報・通信業,688200,775.0
7,アララ,4015,情報・通信業,489500,828.0
8,ヤプリ,4168,情報・通信業,684400,1690.0
9,ＨＥＮＮＧＥ,4475,情報・通信業,729500,1125.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,429200,852.0
1,セキュアヴェイル,3042,情報・通信業,1885800,274.0
2,海帆,3133,小売業,7593800,598.0
3,ドリコム,3793,情報・通信業,589500,586.0
4,Ａｉｍｉｎｇ,3911,情報・通信業,1710400,383.0
5,プレイド,4165,情報・通信業,758500,706.0
6,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,801500,1216.0
7,ココナラ,4176,情報・通信業,287800,292.0
8,スパイダープラス,4192,情報・通信業,275100,661.0
9,エクサウィザーズ,4259,情報・通信業,630000,387.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,429200,852.0
1,Ａｉｍｉｎｇ,3911,情報・通信業,1710400,383.0
2,ココナラ,4176,情報・通信業,287800,292.0
3,スパイダープラス,4192,情報・通信業,275100,661.0
4,エクサウィザーズ,4259,情報・通信業,630000,387.0
5,カルナバイオサイエンス,4572,医薬品,386800,746.0
6,キャンバス,4575,医薬品,218800,944.0
7,カイオム・バイオサイエンス,4583,医薬品,309800,152.0
8,ステムリム,4599,医薬品,220000,831.0
9,モダリス,4883,医薬品,392600,150.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[92649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9967]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,832900,405.0
1,クシム,2345,情報・通信業,1398800,334.0
2,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,911000,692.0
3,出前館,2484,情報・通信業,712600,381.0
4,イメージ ワン,2667,卸売業,509600,610.0
5,アップルインターナショナル,2788,卸売業,856900,553.0
6,ポラリス・ホールディングス,3010,サービス業,1273000,196.0
7,ビューティ花壇,3041,卸売業,1409100,1798.0
8,クラウディアホールディングス,3607,繊維製品,533200,662.0
9,ジェイ・エスコムホールディングス,3779,情報・通信業,1085700,165.0


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
1,クシム,2345,情報・通信業,1398800,334.0
2,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,911000,692.0
8,クラウディアホールディングス,3607,繊維製品,533200,662.0
14,ジオスター,5282,ガラス・土石製品,2836300,306.0
16,ヒーハイスト,6433,機械,1002500,271.0
18,シライ電子工業,6658,電気機器,520800,669.0
19,フェローテックホールディングス,6890,電気機器,840000,3015.0
24,アースインフィニティ,7692,小売業,1603100,175.0
27,ラオックスホールディングス,8202,小売業,1027000,255.0
28,アコム,8572,その他金融業,1897400,359.0


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,3909700,2140.0
1,ベースフード,2936,食料品,638600,638.0
2,ツクルバ,2978,不動産業,890100,1267.0
3,セキュアヴェイル,3042,情報・通信業,1885800,274.0
4,海帆,3133,小売業,7593800,598.0
5,アプリックス,3727,情報・通信業,1835400,223.0
6,ドリコム,3793,情報・通信業,589500,586.0
7,イメージ情報開発,3803,情報・通信業,832600,537.0
8,Ａｉｍｉｎｇ,3911,情報・通信業,1710400,383.0
9,エコモット,3987,情報・通信業,602000,768.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
3,セキュアヴェイル,3042,情報・通信業,1885800,274.0
4,海帆,3133,小売業,7593800,598.0
6,ドリコム,3793,情報・通信業,589500,586.0
11,プレイド,4165,情報・通信業,758500,706.0
13,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,801500,1216.0
15,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,9419400,979.0
18,免疫生物研究所,4570,医薬品,634100,697.0
20,サンバイオ,4592,医薬品,605500,648.0
21,ヘリオス,4593,医薬品,584300,253.0
22,ブライトパス・バイオ,4594,医薬品,526600,130.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')